In [52]:
%useLatestDescriptors
%use dataframe
%use kandy
%use ktor-client

@file:DependsOn("org.jetbrains.kotlinx:kotlinx-io-core:0.8.0")
@file:DependsOn("org.apache.commons:commons-compress:1.28.0")

In [2]:
import kotlinx.io.asInputStream
import kotlinx.io.buffered
import kotlinx.io.files.Path
import kotlinx.io.files.SystemFileSystem
import org.apache.commons.compress.archivers.tar.TarArchiveInputStream
import org.apache.commons.compress.compressors.gzip.GzipCompressorInputStream

private val remoteDataUrl = "https://github.com/ageron/handson-ml/raw/master/datasets"
private val housingUrl = "$remoteDataUrl/housing/housing.tgz"

private val localDataDir = "/Users/markn/IdeaProjects/ml-project/datasets"
private val housingTgzFile = "housing.tgz"
private val housingCsvFile = "housing.csv"

val housingTgzPath = Path("$localDataDir/$housingTgzFile")
val housingCsvPath = Path("$localDataDir/$housingCsvFile")

fun loadHousingTgz() : Path {
    val data = http.get(housingUrl).readBytes()
    SystemFileSystem.createDirectories(Path(localDataDir))
    SystemFileSystem.sink(housingTgzPath).buffered().use {
        it.write(data)
    }
    return housingTgzPath
}

fun extractHousingCsv(path: Path) : Path {
    SystemFileSystem.source(path).buffered().use { source ->
        GzipCompressorInputStream(source.asInputStream()).use { gins ->
            TarArchiveInputStream(gins).use { tais ->
                var entry = tais.nextEntry
                while (entry != null) {
                    if (entry.name == housingCsvFile) {
                        SystemFileSystem.sink(housingCsvPath).buffered().use { sink ->
                            sink.write(tais.readBytes())
                        }
                    }
                    entry = tais.nextEntry
                }
            }
        }
    }
    return housingCsvPath
}

In [ ]:
val housingTgz = loadHousingTgz()
extractHousingCsv(housingTgz)

In [3]:
val housing = DataFrame.readCsv(File(housingCsvPath.toString()))

In [64]:
val plots = listOf(
    housing.plot { histogram(longitude) },
    housing.plot { histogram(latitude) },
    housing.plot { histogram(housing_median_age) },

    housing.plot { histogram(total_rooms) },
    housing.plot { histogram(total_bedrooms.dropNulls()) },
    housing.plot { histogram(population) },

    housing.plot { histogram(households) },
    housing.plot { histogram(median_income) },
    housing.plot { histogram(median_house_value) },
)
plotGrid(plots, nCol = 3)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3m9Ny1"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"layout":{
"name":"grid",
"ncol":3,
"nrow":3,
"fit":true,
"align":false
},
"figures":[{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[151.0,47.0,219.0,2117.0,2792.0,1442.0,752.0,523.0,610.0,617.0,666.0,3366.0,3950.0,1537.0,1518.0,151.0,54.0,97.0,4.0,27.0],
"x":[-123.99399999999991,-123.4919999999999,-122.9899999999999,-122.48799999999991,-121.9859999999999,-121.4839999999999,-120.98199999999991,-120.4799999999999,-119.97799999999991,-119.47599999999991,-118.9739999999999,-118.47199999999991,-117.96999999999991,-117.4679999999999,-116.96599999999992,-116.46399999999991,-115.9619999999999,-115.45999999999992,-114.95799999999991,-114.4559999999999]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"longitude",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"169"
},{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[1235.0,495.0,2522.0,6386.0,527.0,248.0,362.0,182.0,467.0,781.0,1823.0,2888.0,1022.0,745.0,380.0,165.0,103.0,219.0,42.0,48.0],
"x":[32.699750000000016,33.17025000000002,33.64075000000002,34.11125000000001,34.581750000000014,35.052250000000015,35.522750000000016,35.99325000000002,36.46375000000002,36.93425000000002,37.404750000000014,37.875250000000015,38.34575000000002,38.81625000000002,39.28675000000002,39.75725000000002,40.22775000000002,40.698250000000016,41.16875000000002,41.63925000000002]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"latitude",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"170"
},{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[1.275,3.8249999999999997,6.375,8.924999999999999,11.475,14.024999999999999,16.575,19.125,21.674999999999997,24.225,26.775,29.324999999999996,31.875,34.425,36.974999999999994,39.525,42.074999999999996,44.625,47.175,49.724999999999994],
"count":[62.0,497.0,335.0,675.0,492.0,1226.0,1469.0,1537.0,845.0,1492.0,1578.0,937.0,1638.0,1513.0,1793.0,673.0,1017.0,650.0,620.0,1591.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"housing_median_age",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"171"
},{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[982.95,2948.8500000000004,4914.75,6880.650000000001,8846.55,10812.45,12778.350000000002,14744.25,16710.15,18676.050000000003,20641.95,22607.850000000002,24573.75,26539.65,28505.550000000003,30471.45,32437.350000000002,34403.25,36369.15,38335.05],
"count":[9157.0,8366.0,1935.0,625.0,250.0,122.0,63.0,40.0,32.0,17.0,12.0,4.0,5.0,2.0,3.0,3.0,2.0,0.0,0.0,2.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column"

In [13]:
val housingExt = housing.add("income_cat") { ceil(median_income / 1.5).coerceIn(minimumValue = 1.0, maximumValue = 5.0) }

In [47]:
fun <T> DataFrame<T>.stratSplit(
    random: Random = Random(42),
    fraction: Float,
    cols: ColumnsSelectionDsl<T>.(ColumnsSelectionDsl<T>) -> ColumnsResolver<*>,
): DataFrame<T> = groupBy(moveToTop = true, cols)
    .mapToFrames {
        val sampleSize = (group.rowsCount() * fraction).toInt().coerceAtLeast(1)
        group.shuffle(random).take(sampleSize)
    }
    .concat()

In [43]:
import kotlin.random.Random

val housingTrainRandom = housingExt.shuffle(Random(42)).take((housing.rowsCount() * 0.2).roundToInt())
val housingTrainStrat = housingExt.stratSplit(fraction = 0.2f) { income_cat }

In [44]:
val plots = listOf(
    housingExt.plot { histogram(income_cat) },
    housingTrainRandom.plot { histogram(income_cat) },
    housingTrainStrat.plot { histogram(income_cat) },
)
plotGrid(plots, nCol = 3)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="g9FrCv"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"layout":{
"name":"grid",
"ncol":3,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[822.0,0.0,0.0,0.0,0.0,6581.0,0.0,0.0,0.0,0.0,7236.0,0.0,0.0,0.0,0.0,3639.0,0.0,0.0,0.0,2362.0],
"x":[1.1,1.2999999999999998,1.5,1.7000000000000002,1.9,2.1,2.3000000000000003,2.5,2.7,2.9,3.1,3.3000000000000003,3.5,3.7,3.9000000000000004,4.1,4.300000000000001,4.5,4.7,4.9]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"income_cat",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"73"
},{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[153.0,0.0,0.0,0.0,0.0,1269.0,0.0,0.0,0.0,0.0,1481.0,0.0,0.0,0.0,0.0,757.0,0.0,0.0,0.0,468.0],
"x":[1.1,1.2999999999999998,1.5,1.7000000000000002,1.9,2.1,2.3000000000000003,2.5,2.7,2.9,3.1,3.3000000000000003,3.5,3.7,3.9000000000000004,4.1,4.300000000000001,4.5,4.7,4.9]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"income_cat",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"74"
},{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[164.0,0.0,0.0,0.0,0.0,1316.0,0.0,0.0,0.0,0.0,1447.0,0.0,0.0,0.0,0.0,727.0,0.0,0.0,0.0,472.0],
"x":[1.1,1.2999999999999998,1.5,1.7000000000000002,1.9,2.1,2.3000000000000003,2.5,2.7,2.9,3.1,3.3000000000000003,3.5,3.7,3.9000000000000004,4.1,4.300000000000001,4.5,4.7,4.9]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"income_cat",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"75"
}],
"kind":"subplots"
};
 var containerDiv = document.getElementById("g9FrCv");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 7,000 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 income_cat 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 


In [57]:
val fractionsFull = housingExt
    .groupBy { income_cat }
    .count()
    .add("fraction") { it["count"] as Int / housingExt.count().toDouble() }
    .select("income_cat", "fraction")

val fractionsRandom = housingTrainRandom
    .groupBy { income_cat }
    .count()
    .add("fraction") { it["count"] as Int / housingTrainRandom.count().toDouble() }
    .select("income_cat", "fraction")

val fractionsStrat = housingTrainStrat
    .groupBy { income_cat }
    .count()
    .add("fraction") { it["count"] as Int / housingTrainStrat.count().toDouble() }
    .select("income_cat", "fraction")

In [60]:
val fractionDf = fractionsFull
    .sortBy { income_cat }
    .rename { fraction }.into { "overall" }
    .join(fractionsRandom.rename { fraction }.into { "random" })
    .join(fractionsStrat.rename { fraction }.into { "strat" })
    .add("random_error") { ((it["random"] as Double - it["overall"] as Double) / it["overall"] as Double) * 100 }
    .add("strat_error") { ((it["strat"] as Double - it["overall"] as Double) / it["overall"] as Double) * 100 }

println(fractionDf)

   income_cat  overall   random    strat random_error strat_error
 0        1,0 0,039826 0,037064 0,039748    -6,934307   -0,194954
 1        2,0 0,318847 0,307413 0,318953    -3,586081    0,033270
 2        3,0 0,350581 0,358769 0,350703     2,335544    0,034647
 3        4,0 0,176308 0,183382 0,176200     4,012091   -0,061500
 4        5,0 0,114438 0,113372 0,114397    -0,931414   -0,036242

